# 📘 CityAI – Cerca de textos a les subvencions públiques

Aquest notebook utilitza dades obertes de l’Ajuntament de Barcelona per practicar conceptes d’algorísmica. En concret es treballaran els algorismes de cerca de text exacte i altres algorismes de manipulació de cadenes.

## 🧠 Objectius

Aprofundir en els algorismes vistos a teoria, i veure la seva aplicació en un cas real.

## ✍️ Exercici 1: LLegir les dades del fitxer i guardar-les en una llista

L'Ajuntament de Barcelona guarda i publica les dades de les subvencions que concedeix en un fitxer públic. Aquest fitxer té les següents dades:
+ L'entitat municipal que ha concedit la subvenció
+ L'any en què s'ha concedit la subvenció
+ La tipologia de subvenció
+ L'objecte de la subvenció
+ L'import sol.licitat

*Nota*: Hem reduit el fitxer i simplificat les dades per facilitar  treballar-hi. Per aixo treballarem amb el fitxer SubvencionsReduitNetejat.csv

Escriu una funció que llegeixi les dades de SubvencionsReduitNetejat.csv i les guardi en una llista de llistes.

In [5]:

import csv

def llegir_dades(nom_fitxer: str) -> list[list[any]]:
    """
    Aquesta funció llegeix les dades dels fitxers de subvencions de l'Ajuntament 
    de Barcelona i retorna una llista de llistes.
    """
    dades = []

    with open(nom_fitxer, encoding='utf-8') as fitxer:
        lector = csv.reader(fitxer, delimiter=';')
        
        # Si el fitxer té capçalera, la saltem
        primera = next(lector)
        if "Ajuntament" not in primera[0]:
            # si la primera línia era la capçalera
            pass
        else:
            # si no hi ha capçalera, comencem amb la primera
            dades.append([
                primera[0].strip(),
                int(primera[1]),
                primera[2].strip(),
                primera[3].strip(),
                int(float(primera[4]))
            ])

        # Llegim la resta de línies
        for fila in lector:
            if len(fila) < 5:  # ignora línies buides o incorrectes
                continue
            
            entitat = fila[0].strip()
            any_subvencio = int(fila[1])
            tipologia = fila[2].strip()
            objecte = fila[3].strip()
            import_solicitat = int(float(fila[4]))

            dades.append([entitat, any_subvencio, tipologia, objecte, import_solicitat])

    return dades


In [6]:
# ✅ Asserts públics
dades = llegir_dades("SubvencionsReduitNetejat.csv")
assert(dades[5]==['Ajuntament de Barcelona', 2022, 'Convocatoria 2022 Impuls socioeconomic del territori IMPULSEM', 'L estiu Cooperatiu de Sant Marti', 400000])
assert(dades[159]==['Ajuntament de Barcelona', 2017, 'CONVOCATORIA GENERAL DE SUBVENCIONS 2017  SANT ANDREU', 'CULTURA INTERBARRIAL', 12000])

## ✍️ Exercici 2 – Subvencions d'un tema.

Hi ha alguna subvenció relacionada amb els gegants? (cerca exacta per Horspool)

Aplica l'algorisme de Horspool de teoria per buscar alguns temes específics dins dels objectes de  les subvencions. Els temes s'han de trobar tant si estan en majúscula com en minúscula.

Prova de trobar "gegant", "salud", "cultura", "teatre", "infancia"

Si els trobes retorna la fila de dades que els conté i destaca la seva aparació amb ** davant i ** darrera del text trobat.

Per ex. buscar_patro(dades,"gegant") ha de retornar (mira codi Markdown)
[['Ajuntament de Barcelona',
  2019,
  'CONVOCATORIA GENERAL DE SUBVENCIONS 2019. HORTA GUINARDO',
  'Una **gegant**a  Carmela  i els seus cap grossos.',
  46700]]

Pots trobar una explicació detallada de l'algoritme de Hoorspol en aquest vídeo:  
https://www.youtube.com/watch?v=PHXAOKQk2dw  
(també anomenat algoritme de Boyer–Moore–Horspool, ja que és una simplificació de l'algoritme de Boyer-Moore).

In [ ]:
def build_shift_table(patro: str) -> dict:
    """
    Construeix la taula de desplaçament per Horspool (casefold ja aplicat).
    """
    m = len(patro)
    table = {}
    for i in range(m - 1):
        table[patro[i]] = m - 1 - i
    return table

def BoyerMooreHorspool(patro: str, text: str) -> int:
    """
    Retorna l'índex de la primera aparició de patro a text utilitzant
    l'algorisme de Horspool. Tots dos són comparats en mode case-insensitive
    (s'usa .casefold()).
    Si no es troba, retorna -1.
    """
    if not patro:
        return 0
    P = patro.casefold()
    T = text.casefold()
    m = len(P)
    n = len(T)
    if m > n:
        return -1

    shift = build_shift_table(P)
    i = m - 1  # index a T

    while i < n:
        k = 0
        # compare backwards
        while k < m and P[m - 1 - k] == T[i - k]:
            k += 1
        if k == m:
            return i - m + 1
        # caràcter que no coincideix (o el que hi ha a la pos d'inspecció)
        c = T[i]
        d = shift.get(c, m)
        i += d
    return -1

def troba_totes_horspool(patro: str, text: str) -> list:
    """
    Troba totes les posicions de patro dins text (no solapades) utilitzant Horspool.
    Retorna una llista d'índexs d'inici.
    """
    starts = []
    start_pos = 0
    n = len(text)
    while start_pos <= n - len(patro):
        idx = BoyerMooreHorspool(patro, text[start_pos:])
        if idx == -1:
            break
        real_idx = start_pos + idx
        starts.append(real_idx)
        # mouem la posició per seguir cercant a la dreta (per evitar bucle infinit)
        start_pos = real_idx + max(1, len(patro))
    return starts

def highlight_text(text: str, matches: list, L: int) -> str:
    """
    Dona el text original i una llista d'índexs d'inici de coincidències,
    i la longitud L del patró; retorna el text amb cada coincidència envoltada per 
    Gestiona múltiples coincidències ajustant offsets.
    """
    if not matches:
        return text
    parts = []
    last = 0
    offset = 0
    for s in matches:
        parts.append(text[last:s])
        parts.append("**" + text[s:s+L] + "**")
        last = s + L
    parts.append(text[last:])
    return "".join(parts)

def buscar_patro(dades: list, patro: str) -> list:
    """
    Cerca (case-insensitive) dins del camp objectee de cada fila de dades.
    Retorna la llista de files que contenen el patró, on el camp 'objecte' està marcat
    amb ** abans i després de la coincidència.
    Cada fila retornada té la mateixa estructura que a dades però amb l'objecte destacat.
    """
    resultats = []
    if not patro:
        return resultats
    for fila in dades:
        # assumim estructura [entitat, any, tipologia, objecte, import]
        objecte = fila[3]
        # busquem totes les coincidències (basant-nos en casefold)
        matches = troba_totes_horspool(patro, objecte)
        if matches:
            highlighted = highlight_text(objecte, matches, len(patro))
            nova_fila = [fila[0], fila[1], fila[2], highlighted, fila[4]]
            resultats.append(nova_fila)
    return resultats


In [8]:
buscar_patro(dades,"gegant")

[['Ajuntament de Barcelona',
  2019,
  'CONVOCATORIA GENERAL DE SUBVENCIONS 2019  HORTA GUINARDO',
  'DINAMITZACIO COLLA **GEGANT**ERA DE MONTBAU',
  12000],
 ['Ajuntament de Barcelona',
  2019,
  'CONVOCATORIA GENERAL DE SUBVENCIONS 2019  HORTA GUINARDO',
  'Una **Gegant**a  Carmela  i els seus cap grossos',
  46700]]

In [9]:
# ✅ Asserts públics
assert(buscar_patro(dades,"infancia")) == [['Ajuntament de Barcelona',
  2017,
  'AJUTS',
  'Ajut del programa Promocio i participacio **infancia**',
  52290],
 ['Ajuntament de Barcelona',
  2017,
  'AJUTS',
  'Ajut del programa Promocio i participacio **infancia**',
  53070],
 ['Ajuntament de Barcelona',
  2018,
  'AJUTS',
  'Ajut del programa Promocio i participacio **infancia**',
  47850],
 ['Ajuntament de Barcelona',
  2017,
  'AJUTS',
  'Ajut del programa Promocio i participacio **infancia**',
  3960]]

## ✍️ Exercici 3 – Abreviació d'entitats.

Hi ha tantes subvencions que l'ajuntament ha decidit guardar els noms de les entitats que concedeixen els ajuts de forma abreujada, guardant només les seves inicials. Per exemple: Institut Municipal Barcelona Esports es guardarà com IMBE

Escriu una funció que a partir d'una frase sense punts, comes, accents, números ni cap signe de puntuació, imprimeixi l'acrònim corresponent.

In [12]:
def acronim(frase: str) -> str:
    """
    Aquesta funció retorna l'acrònim d'una frase.
    
    Parameters
    ----------
    frase : str
    
    Returns
    -------
    acronim : str
    """
    # Eliminem espais extres i separem les paraules
    paraules = frase.strip().split()
    
    # Agafem la primera lletra de cada paraula i la posem en majúscula
    acronim = ''.join(paraula[0].upper() for paraula in paraules if paraula)
    
    return acronim


def acronim2(frase: str) -> str:
    """
    Versió alternativa de la funció acronim.
    Fa exactament el mateix: retorna les inicials en majúscula.
    """
    return ''.join(paraula[0].upper() for paraula in frase.split() if paraula)


In [13]:
# ✅ Asserts públics
assert(acronim("Institut Municipal Barcelona Esports")) == "IMBE"
assert(acronim2("Institut Municipal Barcelona Esports")) == "IMBE"

## ✍️ Exercici 4 – Alfabet d'aviació

Com que ara hi ha tantes sigles, sovint hi ha confusions per telèfon, i els responsables de l'ajuntament han adoptat l'alfabet de l'aviació per a comunicar-se.

Alfabet d'aviació:
|   |   |   |   |   |
|---|---|---|---|---|
| A | B | C | D | E |
| Alpha | Bravo | Charlie | Delta | Echo |
| F | G | H | I | J |
| Foxtrot | Golf | Hotel | India | Juliet |
| K | L | M | N | O |
| Kilo | Lima | Mike | November | Oscar |
| P | Q | R | S | T |
| Papa | Quebec | Romeo | Sierra | Tango |
| U | V | W | X | Y |
| Uniform | Victor | Whiskey | X-ray | Yankee |
| Z |   |   |   |   |
| Zulu |   |   |   |   |

Escriu una funció que, usant un diccionari, converteixi una cadena de lletres a l'alfabet d'aviació, generant una llista de sortida.

Per exemple: IMBE generarà la llista [India, Mike, Bravo, Echo]

In [ ]:
def aviacio(cadena: str) -> list:
    """
    Aquesta funció converteix una cadena d'entrada a l'alfabet fonètic de l'aviació.
    
    Parameters
    ----------
    cadena : str
    
    Returns
    -------
    traduccio : list
        Llista amb la correspondència de cada lletra segons l'alfabet fonètic.
    """
    # Diccionari de l'alfabet d'aviació (NATO)
    alfabet = {
        'A': 'Alpha', 'B': 'Bravo', 'C': 'Charlie', 'D': 'Delta', 'E': 'Echo',
        'F': 'Foxtrot', 'G': 'Golf', 'H': 'Hotel', 'I': 'India', 'J': 'Juliet',
        'K': 'Kilo', 'L': 'Lima', 'M': 'Maria', 'N': 'November', 'O': 'Oscar',
        'P': 'Papa', 'Q': 'Quebec', 'R': 'Romeo', 'S': 'Sierra', 'T': 'Tango',
        'U': 'Uniform', 'V': 'Victor', 'W': 'Whiskey', 'X': 'X-ray', 'Y': 'Yankee',
        'Z': 'Zulu'
    }
    
    traduccio = []
    for lletra in cadena.upper():  
        if lletra in alfabet:
            traduccio.append(alfabet[lletra])
        else:
            traduccio.append(lletra)  # si no és lletra (p.ex. espai), es deixa igual
    
    return traduccio


In [15]:
# ✅ Asserts públics
assert aviacio('YtH') == ['Yankee', 'Tango', 'Hotel']